## **Data Cleaning**

In this notebook, we’ll go through the **first step of data preprocessing — cleaning the data**

Real-world datasets are rarely perfect. They often contain:

- Missing values (e.g., a person’s age not recorded)

- Duplicates (same record appearing more than once)

- Inconsistent values (typos, wrong formats)

- Outliers (unusual data points that may affect the model)

If we don’t fix these issues, machine learning models may learn incorrect patterns and give poor results.

What we’ll do in this notebook

- Detect and handle missing values

- Remove duplicates

- Identify and treat outliers

- Make the dataset consistent and ready for analysis

By the end, we’ll have a clean, reliable dataset that can be used for further preprocessing and modeling.

In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd

adult = fetch_openml(name="adult", version=2, as_frame=True)

df_adult = adult.frame

df_adult.info()

In [ ]:
df_adult.shape

In [ ]:
# Check first rows
df_adult.head()

It is given in the documentation for the dataset that fnlwgt is a sampling column(not an actual feature). Therefore its better to drop it

In [ ]:
df_adult=df_adult.drop(columns=['fnlwgt'])

### **Handling Missing Values**

In real-world datasets, it’s very common to have some missing or blank entries. For example, in a survey dataset, a person might choose not to answer a question like “income” or “workclass.”

Why does it matter?

- Most machine learning algorithms cannot handle missing values directly.

- If we ignore them, the model may fail to train or give unreliable results.

- Too many missing values can reduce the amount of usable data, which weakens the model.

So, instead of just leaving them as blanks, we use different strategies to handle them.

**How do we handle them?**

There is no “one-size-fits-all” method—it depends on the amount and type of missing data. Some common strategies are:

- Drop rows or columns

    - If only a very small number of rows are missing values, we can remove them without losing much information.

    - Similarly, if an entire column has too many missing values, sometimes it’s better to drop it.

- Imputation (filling in values)

    - Numerical features: Fill missing values with the mean or median of the column.

        - Mean is good when the data is evenly distributed.

        - Median is better if the column has outliers.

    - Categorical features: Replace missing values with the most frequent (mode) value.

- Special category for missing values

    - For categorical columns, we can add a new label like "Unknown" or "Missing".

    - This is useful when missingness itself might carry information (e.g., people not reporting occupation could have a pattern).

In [ ]:
df_adult.isnull().sum()

Applying it to this dataset

- native-country → Has about 857 missing values (~2% of data). This is small, so we can safely fill it with the most common country (mode).

- workclass and occupation → Each has ~6% missing values. Instead of dropping these rows (which would throw away a lot of data), we can fill them with a new category "Unknown". This way, we don’t lose information, and the model can learn from it.

In [ ]:
df_adult['workclass'] = df_adult['workclass'].astype('object')
df_adult['occupation'] = df_adult['occupation'].astype('object')

df_adult['workclass'] = df_adult['workclass'].fillna('Unknown')
df_adult['occupation'] = df_adult['occupation'].fillna('Unknown')

In [ ]:
df_adult['native-country'] = df_adult['native-country'].fillna(df_adult['native-country'].mode()[0])

### **Removing Duplicate Values**

Sometimes, datasets contain duplicate rows, meaning the exact same entry appears more than once.
For example, if two identical records of the same person are present, the dataset is giving them more importance than others.

**Why does it matter?**

- Duplicates can bias the model by giving extra weight to certain records.

- This reduces fairness and may affect predictions.

- Cleaning duplicates ensures every observation is unique and treated equally.

**How do we handle them?**

- Detect and remove duplicates.

- Keep only the first occurrence (or the most relevant one if the dataset has timestamps).

In [ ]:
df_adult.drop_duplicates(inplace=True)

In [ ]:
df_adult.shape

### **Detecting Outliers**

**What are outliers?**
Outliers are extreme values that don’t fit the normal pattern of data. For example:

- A person working 200 hours per week

- Someone with an income of 10 million in a dataset where most incomes are under 100k

Outliers can appear for two reasons:

- **Errors in data entry or measurement** → e.g., a person’s age recorded as 500.

- **Genuine rare cases** → e.g., someone earning very high income compared to others.

If outliers are errors, they should be removed. But if they are real values, they might still carry important information. That’s why we need to carefully analyze outliers before deciding what to do.

#### **Method 1 : Data Visualization**
The easiest way to spot outliers is by visualizing the data (for example, using a boxplot or histogram).

- A boxplot shows the spread of data and highlights values that fall far from the rest.

- For example, when plotting the age column, we can quickly see if some ages are much higher or lower than most others.

Visualization helps us get an intuition about whether unusual values look like mistakes or valid rare cases.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x=df_adult['age'])
plt.show()

We can see here that people in the age group 80–90 are flagged as outliers. This happens because they are relatively rare compared to the rest of the population.

However, this is not a data entry mistake—it’s simply a valid but less common case (some people really do live and work into their 80s and 90s).

In such situations, we keep the values instead of removing them, since they represent real-world diversity.

#### **Method 2: Using Summary Statistics**
Another common way to detect outliers is by using the **Interquartile Range (IQR)**.
- Calculate the Interquartile Range 

$$
\text{IQR} = Q3 – Q1
$$ 

- Define thresholds:

$$
\text{Lower bound}= Q1-  1.5 X  IQR
$$

$$
\text{Upper bound}=Q3+ 1.5 XIQR
$$

Any value outside this range is considered an outlier

This method is useful because it works for many distributions without making assumptions about the data.

In [ ]:
numeric_cols = df_adult.select_dtypes(include=['int64', 'float64']).columns

for col in numeric_cols:
    Q1 = df_adult[col].quantile(0.25)
    Q3 = df_adult[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df_adult[(df_adult[col] < lower) | (df_adult[col] > upper)]
    print(f"{col}: {outliers.shape[0]} outliers")

**Column-wise Observations**

- **hours-per-week:**
Most people work around 40 hours per week, but some work 60–90. These are not mistakes — they are valid, just less common. We should keep them.

- **education-num:**
This column represents years of education, mapped into categories between 1 and 16. Since the range is fixed, values outside this are unlikely. Outliers here are usually not errors but edge cases.

- **capital-gain:**
Highly skewed — most people have 0, but a few have very high values. These are genuine rare high-income cases. Dropping them would lose important information.

- **capital-loss:**
Same reasoning as capital-gain. Most values are 0, with a few large ones that are valid.

**What to do with outliers?**

- **In many datasets:** Outliers are dropped because they can add noise and reduce model accuracy.

- **In this dataset:** Some “outliers” are actually real rare cases (e.g., high capital gains/losses, long working hours). Dropping them would remove valuable information.

Instead of dropping, we apply a log transformation:

**Log Transformation**

- Log reduces the scale of very large numbers while keeping their relative differences intact.

- Example: instead of values like 0, 5000, 10000, 20000, after log we get much smaller, compressed values.

- This way, the influence of extreme values is reduced, but we don’t lose the data.

**Final Note:** Outlier handling is not about blindly removing unusual values. It’s about understanding the data and deciding whether those values are mistakes or important rare cases. In our case, we keep them and use log transformation to make the model handle them better.

In [ ]:
import numpy as np
import pandas as pd

print(df_adult['capital-gain'].describe())

# Apply log transform
df_adult['capital-gain'] = np.log1p(df_adult['capital-gain'])

print(df_adult['capital-gain'].describe())

In [ ]:
import numpy as np
import pandas as pd

print(df_adult['capital-loss'].describe())

# Apply log transform
df_adult['capital-loss'] = np.log1p(df_adult['capital-loss'])

print(df_adult['capital-loss'].describe())

In [ ]:
df_adult.to_csv("adult_cleaned.csv", index=False)

Now our dataset is cleaned - with missing values handled, duplicates removed and outliers treated- the data is trustworthy. The next step is to perform some **exploratory data analysis** to understand our data and gain insights from it 